In [8]:
import os
import requests
import time

In [9]:
base_url = "https://api.mangadex.org"
languages = ["en"]

In [20]:
def download_chapter(chapter_id, chapter_number):
    
    r = requests.get(f"{base_url}/chapter/{chapter_id}")
    chapter_title = r.json()["data"]["attributes"]["title"]
    #print(chapter_id) 
    #print(chapter_number)
    #print(chapter_title)
    
    print(f"{base_url}/at-home/server/{chapter_id}")
    r = requests.get(f"{base_url}/at-home/server/{chapter_id}")
    r_json = r.json()
    
    
        
    if "errors" in r_json:
        print(r_json)
        # rate exceeded
        if r_json["errors"][0]["status"] == 429:
            time.sleep(30)
            r = requests.get(f"{base_url}/at-home/server/{chapter_id}")
            r_json = r.json()
        else:
            print("For some reason, cannot proccess given chapter.")
            return
            
            

    host = r_json["baseUrl"]
    chapter_hash = r_json["chapter"]["hash"]
    data = r_json["chapter"]["data"]
    data_saver = r_json["chapter"]["dataSaver"]
        
    for (page_number, page) in enumerate(data):
        #page_number = int(page[0:page.find("-")])
        page_url = f"{host}/data/{chapter_hash}/{page}"
        print(page_number)
        print(page_url)
        #r = requests.get(f"{host}/data/{chapter_hash}/{page}")
        
        #with open(f"{folder_path}/{page}", mode="wb") as f:
        #    f.write(r.content)

    print(f"Downloaded {len(data)} pages.")

In [21]:
def get_given_value_for_key(manga, key):
    if key in manga:
        return manga[key]
    else:
        return None

In [22]:
def find_given_relationship(manga, relationship_type):
    return next(item for item in manga["relationships"] if item["type"] == relationship_type)

In [23]:
def download_manga(title):
    # Searching for the id of the given manga
    res = requests.get(
        f"{base_url}/manga",
        params={"title": title, "includes[]": ["author", "manga", "cover_art"]}
    )
        
    if not "data" in res.json():
        print("For some reason, data is not in the first get request.")
        return
    
        
    manga_data = res.json()["data"][0]
    #print(manga_data)

    manga_id = manga_data["id"]
    title = manga_data["attributes"]["title"]["en"]
    description = manga_data["attributes"]["description"]["en"]
    status = manga_data["attributes"]["status"]
    public_demographic = manga_data["attributes"]["publicationDemographic"]
    # if (public_demographic == "shounen" or public_demographic == "shoujo"):
    #     public_demographic = True
    # else:
    #     public_demographic = False
    
    tags = manga_data["attributes"]["tags"]
    for tag in tags:
        if tag["attributes"]["group"] == "genre":
            genre_id = tag["id"]
            genre_name = tag["attributes"]["name"]["en"]
    
    
    author_data = find_given_relationship(manga_data, "author")
    author_id = author_data["id"]
    author_name = author_data["attributes"]["name"]
    if "biography" in author_data["attributes"] and "en" in author_data["attributes"]["biography"]:
        author_biography = author_data["attributes"]["biography"]["en"]
    else:
        author_biography = None
    
    cover_art_data = find_given_relationship(manga_data, "cover_art")
    filename = cover_art_data["attributes"]["fileName"]
    cover_art_url = f"https://uploads.mangadex.org/covers/{manga_id}/{filename}"
        
    res = requests.get(
        f"{base_url}/manga/{manga_id}/aggregate",
        params={"translatedLanguage[]": languages},
    )
            
    volumes = res.json()["volumes"]
    # dict (chapter_number -> chapter_id)
    chapters = []
    
    
    for (volume_number, volume_data) in volumes.items():
        print(volume_number)
        print("=====================================================")
        #print(volume_data)
        
        chapters = volume_data["chapters"]
        for (chapter_number, chapter_data) in chapters.items():
            chapter_id = chapter_data["id"]
            download_chapter(chapter_id, chapter_number)
            

In [ ]:
download_manga("Vinland saga")